In [4]:
# Import modules
import os
import sys
import numpy as np
import nibabel as nib
import pandas as pd
import subprocess

In [2]:
class Command():
    '''
    Creates a command and an empty command list for UNIX command line programs/applications. Primary use and
    use-cases are intended for the subprocess module and its associated classes (i.e. run).
    Attributes:
        command: Command to be performed on the command line
    '''

    def __init__(self):
        '''
        Init doc-string for Command class.
        '''
        pass

    def init_cmd(self, command):
        '''
        Init command function for initializing commands to be used on UNIX command line.
        Arguments:
            command (string): Command to be used. Note: command used must be in system path
        Returns:
            cmd_list (list): Mutable list that can be appended to.
        '''
        self.command = command
        self.cmd_list = [f"{self.command}"]
        return self.cmd_list

In [3]:
os.chdir("tmp")

In [5]:
os.listdir()

['clusters.dscalar.nii', 'label.L.label.gii', 'label.R.label.gii', 'test.csv']

In [6]:
def load_hemi_labels(file,wb_struct,map_number=1):
    '''working doc-string'''
    
    gii_label = 'data.label.gii'
    
    load_label = Command().init_cmd("wb_command"); load_label.append("-cifti-separate")
    
    load_label.append(file)
    load_label.append("COLUMN")
    load_label.append("-label"); load_label.append(wb_struct)
    load_label.append(gii_label)
    
    subprocess.call(load_label)
    
    gifti_img = nib.load(gii_label)
    
    atlas_data = gifti_img.get_arrays_from_intent('NIFTI_INTENT_LABEL')[1-1].data
    atlas_dict = gifti_img.get_labeltable().get_labels_as_dict()
    
    os.remove(gii_label)
    
    return atlas_data,atlas_dict

In [7]:
[atlas_data,atlas_dict] = load_hemi_labels("../cvs_avg35_inMNI152.aparc.32k_fs_LR.dlabel.nii", "CORTEX_RIGHT")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: get_labeltable method deprecated. Use the gifti_img.labeltable property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0


In [8]:
def load_gii_data(file,intent='NIFTI_INTENT_NORMAL'):
    '''working doc-string'''
    
    # Load surface data
    surf_dist_nib = nib.load(file)
    
    # Number of TRs in data
    num_da = surf_dist_nib.numDA
    
    # Read all arrays and concatenate temporally
    array1 = surf_dist_nib.get_arrays_from_intent(intent)[0]
    
    data = array1.data
    
    if num_da >= 1:
        for da in range(1,num_da):
            data = np.vstack((data,surf_dist_nib.get_arrays_from_intent(intent)[da].data))
            
    # Transpose data such that vertices are organized by TR
    data = np.transpose(data)
    
    # If output is 1D, make it 2D
    if len(data.shape) == 1:
        data = data.reshape(data.shape[0],1)
        
    return data

In [9]:
def load_hemi_data(file,wb_struct):
    '''working doc-string'''
    
    gii_data = 'data.func.gii'
    
    load_gii = Command().init_cmd("wb_command"); load_gii.append("-cifti-separate")
    
    load_gii.append(file)
    load_gii.append("COLUMN")
    load_gii.append("-metric"); load_gii.append(wb_struct)
    load_gii.append(gii_data)
    
    subprocess.call(load_gii)
    
    data = load_gii_data(gii_data)
    
    os.remove(gii_data)
    
    return data

In [10]:
cluster_data = load_hemi_data("../clusters.dscalar.nii", "CORTEX_RIGHT")

In [11]:
# load_gii_data(file)

In [12]:
# def get_cluster_indices(cluster_id,clust_atlas):
#     '''working doc-string'''
    
#     clust_idx = np.where(clust_atlas == cluster_id)[0]
    
#     return clust_idx

In [13]:
# def get_overlaping_idx(clust_id1, clust_atlas1, clust_id2, clust_atlas2):
#     '''working doc-string'''
    
#     label1_idx = get_cluster_indices(int(clust_id1), clust_atlas1)
#     label2_idx = get_cluster_indices(int(clust_id2), clust_atlas2)
#     overlap_idx = np.intersect1d(label1_idx,label2_idx)
    
#     return overlap_idx

In [14]:
# def calc_cluster_area(cluster_id, atlas_array, surf_va_array):
#     '''working doc-string'''
    
#     clust_idx = get_cluster_indices(int(cluster_id), atlas_array)
#     area = sum_idx_area(clust_idx, surf_va_array)
    
#     return area

In [15]:
# def sum_idx_area(clust_idx, surf_va):
#     '''working doc-string'''
    
#     area = sum(surf_va[clust_idx])
    
#     return area

In [16]:
# cluster_idx = get_cluster_indices(cluster_data,atlas_data)

In [17]:
# atlas_dict[1]

In [18]:
# cluster_idx

In [19]:
# cluster_data

In [20]:
# for i in cluster_data.astype(int):
#     if i != 0:
#         print(i)

In [21]:
# np.intersect1d(cluster_data,cluster_idx)

In [22]:
# mask = np.where(cluster_data>0,1,cluster_data)

In [23]:
# for i in mask:
#     if i != 0:
#         print(i)

In [24]:
# mask[3]

In [25]:
# atlas_data

In [26]:
# for i in atlas_data[40:80]:
#     print(i)

In [27]:
# atlas_dict[45]

In [28]:
for idx,val in enumerate(cluster_data.astype(int)):
    if cluster_data[idx] == 0:
        # print(f"{idx} is 0")
        atlas_data[idx] = 0

In [30]:
# idx

In [31]:
atlas_data

array([0, 0, 0, ..., 0, 0, 0])

In [32]:
np.unique(atlas_data)

array([ 0, 43, 60, 64])

In [33]:
for i in np.unique(atlas_data)[1:]:
    print(atlas_dict[i])

R_inferiorparietal
R_precuneus
R_superiorparietal


In [34]:
# a = [0,0,1,0,0]
# b = [7,5,4,8,9]

In [35]:
# # order index_val, array_indexed_val
# for idx,i in enumerate(a):
#     print(idx,i)
#     if a[idx] == 0:
#         b[idx] = 0
        
# a,b

In [36]:
# range(0,1,cluster_data.shape[0])

In [37]:
# enumerate(cluster_data)

In [38]:
def get_roi_name(cluster_data,atlas_data,atlas_dict):
    '''working doc-string'''
    
    # for idx,val in enumerate(cluster_data.astype(int)):
    for idx,val in enumerate(cluster_data):
        if cluster_data[idx] == 0:
            atlas_data[idx] = 0
    
    tmp_list = list()
    roi_list = list()
    
    for i in np.unique(atlas_data)[1:]:
        # print(atlas_dict[i])
        tmp_list = atlas_dict[i]
        roi_list.append(tmp_list)
    
    return roi_list

In [39]:
roi_list = get_roi_name(cluster_data,atlas_data,atlas_dict)
roi_list
# get_roi_name([0],[0],atlas_dict)

['R_inferiorparietal', 'R_precuneus', 'R_superiorparietal']

In [40]:
def find_clusters(file,left_surf,right_surf,thresh = 1.77,distance = 20):
    '''working doc-string'''
    
    cii_data = 'clusters.dscalar.nii'
    
    thresh = str(thresh)
    distance = str(distance)
    
    find_cluster = Command().init_cmd("wb_command"); find_cluster.append("-cifti-find-clusters")
    find_cluster.append(file)
    find_cluster.append(thresh); find_cluster.append(distance)
    find_cluster.append(thresh); find_cluster.append(distance)
    find_cluster.append("COLUMN")
    find_cluster.append(cii_data)
    find_cluster.append("-left-surface")
    find_cluster.append(left_surf)
    find_cluster.append("-right-surface")
    find_cluster.append(right_surf)
    
    subprocess.call(find_cluster)
    
    return cii_data

In [41]:
file = "../dr_stage3_ic0012_tfce_tstat_fwep_c2.dscalar.nii"
r_surf = "../S1200.R.midthickness_MSMAll.32k_fs_LR.surf.gii"
l_surf = "../S1200.L.midthickness_MSMAll.32k_fs_LR.surf.gii"

In [42]:
find_clusters(file,l_surf,r_surf)

'clusters.dscalar.nii'

In [171]:
i_dict = {"File":file,
         "ROIs":[roi_list]}

In [172]:
df = pd.DataFrame.from_dict(i_dict,orient='columns')

In [173]:
df

,File,ROIs
0,../dr_stage3_ic0012_tfce_tstat_fwep_c2.dscalar...,"[R_inferiorparietal, R_precuneus, R_superiorpa..."


In [174]:
df.to_csv('test.csv',sep=",",header=True,index=False,mode="w")

In [47]:
def write_spread(file,out_file,roi_list):
    '''
    Writes image filename, dimensions, and acquisition direction to a
    spreadsheet. If the spreadsheet already exists, then it is appended
    to.
    
    Arguments:
        nii_file (nifti file): NifTi image filename with absolute filepath.
        out_file (csv file): Output csv file name and path. This file need not exist at runtime.
        
    Returns: 
        out_file (csv file): Output csv file name and path.
    '''
    
    # Strip csv file extension from output file name
    if '.csv' in out_file:
        out_file = os.path.splitext(out_file)[0]
        out_file = out_file + '.csv'
    elif '.tsv' in out_file:
        out_file = os.path.splitext(out_file)[0]
        out_file = out_file + '.csv'
    elif '.txt' in out_file:
        out_file = os.path.splitext(out_file)[0]
        out_file = out_file + '.csv'
    else:
        pass
    
    # Construct image dictionary
    file = os.path.abspath(file)
    img_dict = {"File":file,
         "ROIs":[roi_list]}
    
    # Create dataframe from image dictionary
    df = pd.DataFrame.from_dict(img_dict,orient='columns')
    
    # Write output CSV file
    if os.path.exists(out_file):
        df.to_csv(out_file, sep=",", header=False, index=False, mode='a')
    else:
        df.to_csv(out_file, sep=",", header=True, index=False, mode='w')
    
    return out_file

In [48]:
write_spread("../clusters.dscalar.nii",'test2.csv',roi_list)

'test2.csv'

In [ ]:
# iterate through L R hemispheres
# Split file into L/R surfaces and volume
# Load label/cluster data and store info

# Nifti file ROIs

In [127]:
file = "../Atlas_ROIs.2.nii.gz"

In [128]:
img = nib.load(file)

In [141]:
# print(img.header)
